### Graph based line merging

In [ ]:
import linegroupping as lg

In [ ]:
in_line = r"I:\Temp\centerline.shp"
out_line_groupped = r"I:\Temp\centerline_grouped.shp"
out_line_merged = r"I:\Temp\centerline_merged.shp"  # line trimmed and merged.

in_poly = r"I:\Temp\footprint_ground.shp"
out_poly = r"I:\Temp\footprint_ground_clean.shp"

# in_line = r"I:\Temp\centerline_S.shp"
# out_line = r"I:\Temp\centerline_grouped_S.shp"

# in_poly = r"I:\Temp\footprint_ground_S.shp"
# out_poly = r"I:\Temp\footprint_ground_clean_S.shp"

In [ ]:
lg_obj = lg.LineGroupping(in_line, in_poly)
lg_obj.run_groupping()
#  save groupped lines
lg_obj.data.to_file(out_line_groupped)

# trim lines and polygons
# merge lines and make sure no MultiLineString exist
lg_obj.run_cleanup()
lg_obj.run_line_merge()
lg_obj.save_file(out_line_merged, out_poly)